In [1]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env
import polars as pl

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)

In [ ]:

from humbldata.toolbox.technical.mandelbrot_channel.model import (
    calc_mandelbrot_channel,
    calc_mandelbrot_channel_historical,
)


data = (
    obb.equity.price.historical(
        ["AAPL"],
        provider="yfinance",
        start_date="2020-01-01",
        end_date="2024-01-01",
        adjustment="splits_and_dividends",
    ).to_polars()
).drop(["dividends", "stock_splits"]).with_columns(pl.lit("AAPL").alias("symbol"))

mandelbrot = calc_mandelbrot_channel_historical(  # noqa: ERA001
        data,
        window="1m",
        rv_adjustment=True,
        rv_method="std",
        rv_grouped_mean=False,
        rs_method="RS",
        live_price=False,
    ).collect()
mandelbrot

# Setting up UserTable

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.standard_models.portfolio.analytics.user_table import UserTableQueryParams
from humbldata.core.utils.openbb_helpers import get_latest_price


symbols = UserTableQueryParams(symbol="AAPL, MSFT, NVDA").symbol
symbols



get_latest_price(symbols)

# Tesing Async


## Get Sector Async

In [6]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_etf_category
import polars as pl

stocks = pl.Series([
    "GOOGL",  # Communication Services
    "AMZN",   # Consumer Discretionary
    "KO",     # Consumer Staples
    "XOM",    # Energy
    "JPM",    # Financials
    "JNJ",    # Health Care
    "HON",    # Industrials
    "AAPL",   # Information Technology
    "LIN",    # Materials
    "PLD",    # Real Estate
    "NEE"     # Utilities
])
test = (await aget_etf_category(symbols=["AAPL", "XLE", "DBA"], provider="yfinance")).collect()
# test2 = (await aget_equity_sector(symbols=stocks, provider="yfinance")).collect()
# test2 = (await aget_equity_sector(symbols=["XLE", "AAPL"], provider="yfinance")).collect()
test

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


symbol,category
str,str
"""AAPL""",null
"""XLE""","""Equity Energy"""
"""DBA""","""Commodities Focused"""


In [13]:
symbols = ["CORN", 'SLV', 'DBA', "XLE", "AAPL", "FXE", "SPY", "QQQ", "BITO", "MAXI", "SATO", "BIL", "LQD", "QAI", "MNA", "CLSE"]
symbols2 = ["AAPL", "FXE", "SPY", "MAXI", "GLD"]
symbols3 = commodity_etf_symbols = ["GLD", "IAU", "SLV", "GLDM", "PDBC", "SGOL", "FTGC", "DBC", "SIVR", "USO", "IAUM", "GSG", "BCI", "PPLT", "COMT", "GLTR", "OUNZ", "BAR", "UNG", "DBA", "AAAU", "DJP", "CMDT", "KRBN", "PALL", "COM", "NBCM", "CMDY", "DBO", "BCD", "KCCA", "CPER", "HGER", "USCI", "DBB", "DBP", "GCC", "WEAT", "BNO", "UGA", "DGP", "IGLD", "COMB", "DBE", "FGDL", "CORN", "USL", "CCRV", "DJCB", "IAUF", "PLTM", "BDRY", "PDBA", "BGLD", "UCIB", "SOYB", "GRN", "PIT", "BCIM", "RENW", "SDCI", "UNL", "SHNY", "DCMT", "TAGS", "CANE", "TMET", "KEUA", "HCOM", "HARD", "EVMT", "USG", "AMPD", "DZZ", "DGZ", "KMET", "CMCI", "USOY", "ZSC", "TILL", "BWET", "DULL", "ZSB", "LNGG", "USOI", "OILK", "SLVO", "GLDI", "BOIL", "KOLD", "AGQ", "GLL", "UGL", "ZSL", "UCO", "SCO"]



df = (await aget_etf_category(symbols=["AAPL", "XLE", "DBA"])).collect()


In [15]:
from humbldata.portfolio.analytics.user_table.helpers import normalize_asset_class


normalized_df = normalize_asset_class(df).select(["symbol", "category"])
normalized_df

symbol,category
str,str
"""AAPL""",null
"""XLE""","""Equity"""
"""DBA""","""Commodity"""


In [33]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_etf_category


df2 = (await aget_etf_category(symbols=["AAPL", "XLE", "DBA"])).collect()
df2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


symbol,category
str,str
"""AAPL""",null
"""XLE""","""Equity Energy"""
"""DBA""","""Commodities Focused"""


In [41]:
%load_ext autoreload
%autoreload 2

from humbldata.core.standard_models.portfolio.analytics.etf_category import ETFCategoryData


ETFCategoryData(df2)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


symbol,category
str,str
"""AAPL""",null
"""XLE""","""Equity Energy"""
"""DBA""","""Commodities Focused"""


In [25]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.user_table.helpers import aget_asset_class_filter


(await aget_asset_class_filter(symbols=["AAPL", "XLE", "DBA"])).collect()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


symbol,asset_class
str,str
"""AAPL""","""Equity"""
"""XLE""","""Equity"""
"""DBA""","""Commodity"""


## Test aget_sector_filter

In [45]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.user_table.helpers import aget_sector_filter


(await aget_sector_filter(symbols=["XLU", "XLE", "XLF", "XLC", "KO", "AAPL"])).collect()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


symbol,sector
str,str
"""KO""","""Consumer Defensive"""
"""AAPL""","""Technology"""
"""XLU""","""Utilities"""
"""XLE""","""Energy"""
"""XLF""","""Financials"""
"""XLC""","""Communication Services"""


# Test aggregate_user_table_data

In [69]:
from humbldata.portfolio.analytics.user_table.helpers import aggregate_user_table_data


(await aggregate_user_table_data(symbols=["XLU", "XLE", "AAPL"])).collect()

date,symbol,bottom_price,recent_price,top_price,sector,asset_class
date,str,f64,f64,f64,str,str
2024-06-28,"""AAPL""",207.189,210.619995,219.3608,"""Technology""","""Equity"""
2024-06-28,"""XLE""",88.4205,91.150002,91.7758,"""Energy""","""Equity"""
2024-06-28,"""XLU""",67.3069,68.139999,69.8705,"""Utilities""","""Equity"""


In [23]:
from humbldata.core.utils.openbb_helpers import aget_latest_price


(await aget_latest_price(symbol=["AAPL", "NVDA", "XLE"])).collect()

last_price,symbol
f64,str
210.62,"""AAPL"""
123.54,"""NVDA"""
90.8,"""XLE"""


In [53]:
etf_data = await aget_etf_category(symbols=["AAPL", "XLE", "DBA"])
(await aget_sector_filter(symbols=["AAPL", "XLE", "DBA"], etf_data=etf_data)).collect()

symbol,sector
str,str
"""AAPL""","""Technology"""
"""XLE""","""Energy"""
"""DBA""","""Commodities Focused"""
